In [1]:
import numpy as np
import scipy as sc
import pandas as pd

In [2]:
def needleman_wunsch(sequence_a, sequence_b, match_score, mismatch_score, gap_score):
    """
    performs needleman wunsch sequence alignment as described in:
    Needleman, Saul B. & Wunsch, Christian D. (1970).
    "A general method applicable to the search for similarities in the amino acid sequence of two proteins".
    Journal of Molecular Biology.
    Args:
    sequence a, sequence b: the two strings to be aligned
    match_score: the score given to a match between two equal characters
    mismatch_score: the score given to a match between two unequal characters
    gap_score: the score given to a match between a character and a gap
    """
    # number of rows in the dynamic programming tables
    m = len(sequence_a) + 2
    # number of columns in the dynamic programming tables
    n = len(sequence_b) + 2
    # the dynamic programming tables
    score_table = np.array([[None] * n for i in range(m)])
    arrow_table = np.array([[None] * n for i in range(m)])
    # the output list of alignments
    alignments = []

    def initialize():
        """
        initializes the dynamic programming tables
        """
        # initialize the score table with sequence headers
        score_table[0, 2:] = list(sequence_b)
        score_table[2:, 0] = list(sequence_a)
        # initialize the score table with base cases
        score_table[1, 1:] = [-i for i in range(m - 1)]
        score_table[1:, 1] = [-i for i in range(n - 1)]
        # initialize the arrow table with base cases
        arrow_table[1, 2:] = [list("←") for i in range(n - 2)]
        arrow_table[2:, 1] = [list("↑") for i in range(m - 2)]
    
    def build_tables():
        """
        populates the dynamic programming tables
        """
        for i in range(2, m):
            for j in range(2, n):
                # insert a gap in sequence_a
                up_score = score_table[i - 1, j] + gap_score
                # insert a gap in sequence_b
                left_score = score_table[i, j - 1] + gap_score
                if score_table[i, 0] == score_table[0, j]:
                    # match equal characters
                    diagonal_score = score_table[i - 1, j - 1] + match_score
                else:
                    # match unequal characters
                    diagonal_score = score_table[i - 1, j - 1] + mismatch_score
                # determine the optimal score and store it in the score table
                max_score = max(up_score, left_score, diagonal_score)
                score_table[i, j] = max_score
                # determine which arrows describe the optimal step, not necesssarily unique
                arrows = []
                if up_score == max_score:
                    arrows += "↑"
                if left_score == max_score:
                    arrows += "←"
                if diagonal_score == max_score:
                    arrows += "↖"
                # put the list of arrows into the arrow table
                arrow_table[i, j] = arrows

    def find_alignments():
        """
        use the arrow_table to construct the alignments
        we start at the bottom right and follow the arrows back to (1,1)
        a stack is used to facilitate backtracking so that all optimal alignments are found
        """
        alignment_a = []
        alignment_b = []
        i = m - 1
        j = n - 1
        stack = []

        while True:
            # if we reach (1,1) then the current alignment is complete
            if (i, j) == (1, 1):
                # current alignment is complete so add it to the list of alignments
                alignments.append(("".join(alignment_a), "".join(alignment_b)))
                # check if there are any backtracking states in the stack
                if stack == []:
                    # if not then then all alignments have been found and we are done
                    break
                else:
                    # stack is not empty so we backtrack
                    (i, j, arrows, alignment_a, alignment_b) = stack.pop()
            else:
                # general case, copy arrows for this step from from arrow_table
                arrows = list(arrow_table[i, j])
            # process first arrow for this step
            # push the remaining arrows, if any, on to the stack for later backtracking
            arrow = arrows.pop()
            if arrows != []:
                # save this state for later backtracking
                stack.append((i, j, arrows, list(alignment_a), list(alignment_b)))
            # process current arrow
            if arrow == "↖":
                # match the current position in both sequences
                alignment_a.insert(0, sequence_a[i - 2])
                alignment_b.insert(0, sequence_b[j - 2])
                # follow the arrow with a diagonal step
                i = i - 1
                j = j - 1
            elif arrow == "↑":
                # match current position in sequence a with a gap in sequence b
                alignment_b.insert(0, " ")
                alignment_a.insert(0, sequence_a[i - 2])
                # follow the arrow upwards
                i = i - 1
            elif arrow == "←":
                # match current position in sequence b with a gap in sequence a
                alignment_b.insert(0, sequence_b[j - 2])
                alignment_a.insert(0, " ")
                # follow the arrow to the left
                j = j - 1

    initialize()
    build_tables()
    find_alignments()
    # the optimal alignment score is found in the bottom right of the score table
    alignment_score = score_table[m - 1, n - 1]
    return (alignment_score, alignments)

In [41]:
data=pd.read_csv("/Users/carloswertcarvajal/Downloads/trainproteins.csv",delimiter=",")
init=data['peptides'].values
init=list(dict.fromkeys(init))
score=np.zeros([len(init),len(init)])
for n in range(0,len(init)):
    print(n)
    for m in range(0,len(init)):
        if m==n:
            pass
        else:
            score[n,m] , align = needleman_wunsch(init[n],init[m],1,-1,-1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [91]:
arrays=np.where(score>6)
listx = list(zip(*arrays))
c=0
for m in range(0,len(listx)):
    indx = listx[m]
    rev = (indx[1],indx[0])
    listx.remove(rev)
    print(init[indx[0]],init[indx[1]],indx)
    c+=1

KSVEFDMS_ KSVEFDMSH (1, 109)
NTAIAKCNL NTALAKCNL (14, 600)
LNTRRRQL_ LNTRRRQLL (40, 407)
DIYKGVYQ_ DIYKGVYQF (46, 831)
LTPIFSDL_ LTPIFSDLL (49, 157)
VLLEFQSHL VLLEFQSH_ (62, 362)
KTFDHTLM_ KTFDHTLMS (65, 147)
FAPKNDPAL FAPKNLPAL (104, 396)
FAPKNDPAL FAPKNAPAL (104, 608)
VSSIFLHL_ VSSIFLHLL (106, 796)
GLKGPDIY_ GLKGPDIYK (133, 756)
YVPMPCMI_ YVPMPCMIN (148, 189)
HDAEFCDM_ HDAEFCDML (160, 873)
VSRPTTVV_ VSRPTTVL_ (163, 178)
VSRPTTVV_ VGRPTTVV_ (163, 211)
VSRPTTVV_ VSKPTTVV_ (163, 271)
VSRPTTVV_ VSRPTAVV_ (163, 386)
VSRPTTVV_ VSRPTTVM_ (163, 478)
VSRPTTVV_ VLRPTTVV_ (163, 582)
RGYVYQ___ RGYVYQG__ (166, 290)
VSRPTTVL_ VSRPTTVM_ (178, 478)
LSHHFTLV_ LSHHFTVV_ (181, 354)
LSHHFTLV_ LSHYFTLV_ (181, 492)
IYKGVYQF_ IYKGVYQFK (190, 487)
ITRPTTVV_ ITRPTTLV_ (196, 832)
FAPKNYPAD FAPKNYPA_ (198, 236)
FAPKNYPAD FAPKNYPAR (198, 292)
VGRPTTVV_ VLRPTTVV_ (211, 582)
VNPHSTSLI VNPHSTSL_ (213, 713)
TLMSIVSS_ TLMSIVSSL (216, 418)
FAPKNYPA_ FAPKNYPAR (236, 292)
LKGPDIYKG LKGPDIYK_ (240, 253)
NKKTFDHTL NKKTFD

IndexError: list index out of range

In [3]:
s1 = 'LGITYDGM'
s2 = 'MADSHNTQYCSLQESAQAQQELDNDQETMETSEEEEDTTTSNKVYGSGIPSPPQSPQRAYSPCVALASIPDSPSEEASIKGSGGLEDPLYLLHNAQNTKVYDLVDFLVLNYQMKAFTTKAEMLESIGREYEEYYPLIFSEASECLKMVFGLDMVEVDPSVHSYILVTALGITYDGMMTDVLGMPKTGILIAVLSVIFMKGNYVSEEIIWEMVNNIGLCGGRDPYIHKDPRKLISEEFVQEGCLKYRQVPNSDPPSYGFLWGPRAFAETSKMKVLQFFASINKTHPRAYPEKYAEALQDEIDRTKAWILNRCSNSSDLLTF'
alignment_score, alignments=needleman_wunsch(s1, s2, 1, -1, -1)

ValueError: cannot copy sequence with size 9 to array axis with dimension 321